In [8]:
from itertools import product
import numpy as np
import itertools
from itertools import combinations
from copy import deepcopy
from operator import add,mul
from collections import defaultdict
from collections import Counter
from sage.combinat.cartesian_product import CartesianProduct_iters
import time

#Se halla la matriz de Gram del lattice n-dimensional en términos de los parámetros de Selling.

def M(n):    
    M = [[0]*(n)]*(n)
    for i in range(n):
        M[i] = [-var('p%d%d' % tuple(sorted((i+1,j+1)))) for j in range(n)]
        M[i][i] = sum([var('p%d%d' % tuple(sorted((i+1,k)))) for k in range(0,n+1)]) - var('p%d%d' % (i+1,i+1))
        
    return(Matrix(M))


#Calcula el determinante de la matriz de Gram.


def d(n):
    lis=list(combinations(list(range(n+1)), 2))
    c=0
    for j in lis:
        i=list(j)
        c+=var('D%d%d' % tuple(sorted((i[0],i[1]))))*var('p%d%d' % tuple(sorted((i[0],i[1]))))
    return c/n


def G_conj(n):
    """R"""
    
    G = 0
    
    Ck = C_k(n+1)
    
    for g in Ck:
        k = Graph(g,multiedges=True).girth()
        if k == 2:
            G += mono(g)
        if k > 2:
            G += 2*(k-1)*mono(g)
    return(expand(G))


def mono(g):
    """R"""

    mono = 1
    for e in g:
        mono = mono*var('p%d%d' % (sorted(e)[0],sorted(e)[1]))
    return(mono)


#Calcula los árboles generadores del grafo completo Kn.

def C_k(n):
    
    P = prufer(n)
    
    C = []
    
    for ps in P:
        for i in range(n):
            for j in range(i+1,n):
                c = prufer_to_tree(ps)
                c.append([i,j])
                if sorted(c) not in C:
                    C.append(sorted(c))
        
    return(C)



#Calcula el árbol asociado a una secuencia de Prüfer, como una lista de aristas.

def prufer_to_tree(ps):    
    n = len(ps)+2
    ps = list(ps)
    N = list(range(n))
    T = []
    while len(N) > 2:
        j = min(set(N)-set(ps))
        T.append(sorted([j,ps[0]]))
        N.remove(j)
        ps.remove(ps[0])
    T.append([N[0],N[1]])
    return(T)

#Calcula todas las secuencias de Prüfer de longitud n-2

def prufer(n):     
    return(itertools.product(range(n), repeat=n-2))


In [9]:
#vectores de la superbase
dimen = 2
vectors = dimen+1
v  =[]
for i in range(vectors):
    v=v+[var('v{}'.format(i))]
    
#parámetros de selling
r=[]
for i in range(len(v)):
    for j in range(len(v)):
        if i<j:
            r=r+[var('p{}{}'.format(i,j))]

#Algunas condiciones para los parámetros de Selling.           
for i in range(len(v)):
    for j in range(len(v)):
        if i<j:
            assume(var('p{}{}'.format(i,j))>0)
    
for i in range(len(v)):
    for j in range(len(v)):
        if i<j:
            assume(var('p{}{}'.format(i,j)),'real')
        
#La lista de las y-coordenadas.
y=[]
for i in range(len(v)):
    y = y + [var('y{}'.format(i))]  
    
#Definición del producto punto de los vectores de Voronoi en términos de los parámetros de Selling.
def parametrosselling(x,y):
    for i in range(len(v)):
        for j in range(len(v)):
            if i!=j:
                while x==v[i] and y==v[j]:
                    return -var('p{}{}'.format(min(i,j),max(i,j)))               
            else:
                while x==v[i] and y==v[j]:
                    s=0                    
                    for l in range(len(v)):
                        if l!=i:
                            s=s+var('p{}{}'.format(min(i,l),max(i,l)))
                    return s   
                
#Matriz de Gram.

P=matrix(SR,len(v))
for i in range(len(v)):
    for j in range(len(v)):
        if i==j:
            P[i,j]=parametrosselling(v[i],v[i])
        else:
            P[i,j]=parametrosselling(v[i],v[j])
            P[j,i]=P[i,j]
R = range(len(v) )
A=P[ [ k for k in R if k not in [len(v)-1] ] ,
   [ k for k in R if k not in [len(v)-1] ] ]
       

D=(A.det()).factor()


In [10]:
# k número de árboles generadores.

# D es el determinante del lattice.

# l es el multiplicador de Lagrange.


k = (dimen + 1)^(dimen - 1) 

#La función G a optimizar bajo la restricción F.

var ('l')

F = D - k
L = G_conj(dimen) - l*F

#Sistema de ecuaciones de las derivadas de L:

varlist = r+[l]

eqlist = [diff(L,varlist[v]) for v in range(len(varlist))]

#solución del sistema.

sol = solve(eqlist,varlist)


In [11]:
# Una solución del sistema cuando todos los parámetros de Selling son 1:

solucion=[r[i] == 1 for i in range(len(r))]
solucion

#Verificación de que solucion satistace el sistema de escuaciones:
eqlist1=[]
for i in range(len(eqlist)):
    eqlist1.append(eqlist[i].subs(solucion))
sol = solve(eqlist1,l)
print (sol)

solucion1 = solucion + [sol[0][0]]

if all(eqlist[m].subs(solucion1) == 0 for m in range(len(eqlist))) == True:
          print (solucion , 'es solucion del sistema.')

[
[l == 5]
]
[p01 == 1, p02 == 1, p12 == 1] es solucion del sistema.


In [12]:
#La matriz Hessiana.

H1 = L.hessian()


#constante del denominador de G(L).

t = 12*dimen*k^((dimen+1)/dimen) 


#Hay un mínimo local cuando todos los parámetros de Selling son 1.

H = H1.subs(solucion1)
detsub = []
for j in range(3,binomial(dimen+1,2)+2):
    T = H[[k for k in range(j)],[k for k in range(j)]]
    detsub.append(T.subs(solucion1).n(prec=200).det())
if all(-1 == sgn(detsub[m]) for m in range(len(detsub))) == True:
        print ('Un mínimo local ocurre en: ', solucion)
        print ('Un mínimo local es: ', (G_conj(dimen).subs(solucion)/t).n(prec=30))
else:
    print ('No es un mínimo local.')

Un mínimo local ocurre en:  [p01 == 1, p02 == 1, p12 == 1]
Un mínimo local es:  0.080187537
